In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr, colorcet
import numpy as np

from xpublish.routers import XYZFactory
from xpublish.utils.ows import DataShader, MatplotLib

In [3]:
ds = xr.tutorial.open_dataset(
    "air_temperature", chunks=dict(lat=5, lon=5),
)

In [4]:
ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
ds.rio.write_crs(4326, inplace=True)

<xarray.Dataset>
Dimensions:      (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat          (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon          (lon) float32 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time         (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
    spatial_ref  int64 0
Data variables:
    air          (time, lat, lon) float32 dask.array<chunksize=(2920, 5, 5), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [5]:
ds = ds.rio.reproject(
     #epsg: 3857
     dst_crs= "+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs" 
     )

In [6]:
crs_epsg =  3857

In [7]:
render = DataShader(
                      aggregation={}, 
                      color_mapping={ 
                                "cmap": colorcet.coolwarm,
                                "how": "linear", 
                                "span": [float(np.nanmin(ds["air"].values)), 
                                         float(np.nanmax(ds["air"].values))], 
                                "alpha": 180 }
                        )

In [8]:
xyz_router = XYZFactory(
                       crs_epsg= crs_epsg,
                       render= render
)

In [9]:
ds.rest(routers=[xyz_router])

In [10]:
import nest_asyncio 
nest_asyncio.apply()

In [ ]:
ds.rest.serve(host='127.0.0.1',port=9003,log_level='info')